In [1]:
import glob
import pandas as pd
import os, sys
from typing import List, Callable
import numpy as np

if not "../" in sys.path:
    sys.path.insert(0,"../")
from scripts.qsub_base import Base

In [2]:

def gather_reference_quantification(
        quantification_dir: str="data/simulated_data/quantification/", 
        expected_labels: List[str]=None):
    
    summary_files = glob.glob(os.path.join(quantification_dir, "*/cmseq_summation.tsv"))
    summary_labels = [x.rsplit("/",2)[1] for x in summary_files]
    
    if expected_labels:
        if not set(expected_labels) == set(summary_labels):
            err_msg = f"\
expected summary files != found summary files.\n\
Expected labels({len(expected_labels)}):{expected_labels}\n\
Found labels({len(summary_labels)}):{summary_labels}\n\
Missing expected: {set(expected_labels).difference(set(summary_labels))}"
            raise ValueError(err_msg)
    
    dataframes = []
    for file, label in zip(summary_files, summary_labels):
        readsGB = float(label[:-2].replace("_", "."))
        df = pd.read_csv(file, sep="\t")
        df["readsGB"] = readsGB
        dataframes.append(df)        
    print(f"Gathered summary from ({len(summary_labels)}): {summary_labels}", file=sys.stderr)
    return pd.concat(dataframes)

In [7]:
def expected_count_eq_ratio(readsGB:List[int], input_genomes_fasta: str = "data/simulated_data/input_genomes/combined.fa"):
    total_length = 0
    for line in open(input_genomes_fasta, "rb"):
        if line[0] == 62: #bit for '>'
            #print(total_length)
            #print(line)
            continue
        total_length += len(line.strip())
    print(f"Combined genome size ({total_length})bp", file=sys.stderr)
    expected_counts = [(x*10**9)/total_length for x in readsGB]
    return expected_counts

In [121]:

def calculate_error_metrics(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()
    
    fp_inputcombined = "../data/simulated_data/input_genomes/combined.fa"
    df_out["expected_value"] = expected_count_eq_ratio(df["readsGB"], fp_inputcombined)
    df_out["expected_value_int"] = df_out["expected_value"].round(0)
    
    df_out["err"] = df_out["Depth median"] - df_out["expected_value"]
    df_out["AE"] = np.abs(df_out["err"])
    df_out["RAE"] = df_out["AE"]/df_out["expected_value"]
    
    #standardised residuals:
    df_out["std_res"] = df_out["err"] / np.std(df_out["err"], ddof=2)
    #df_out["RAE_int"] = np.abs(df_out["expected_value_int"].values-df_out["Depth median"])/df_out["expected_value_int"]
    
    return df_out
    #df_out["abs_err"] = 

#df_summary = calculate_error_metrics(df_ref_quant)

In [122]:
# Run analysis:
# Which runs are we expecting to see:
gbs_to_run = np.round([x*10**-y for x in range(1,10) for y in (1,2,3)] + [1], 3) #Note rounding should equal max powering
gbs_to_run.sort()
expected_labels = [Base.gen_prefix(gb) for gb in gbs_to_run]
expected_labels

df_ref_quant = gather_reference_quantification("../data/simulated_data/quantification/", expected_labels)
df_summary = calculate_error_metrics(df_ref_quant)


Gathered summary from (28): ['0_5GB', '0_9GB', '0_01GB', '0_07GB', '0_02GB', '0_09GB', '0_007GB', '0_006GB', '0_6GB', '0_08GB', '0_04GB', '0_05GB', '1_0GB', '0_03GB', '0_004GB', '0_002GB', '0_009GB', '0_7GB', '0_8GB', '0_2GB', '0_1GB', '0_001GB', '0_3GB', '0_005GB', '0_008GB', '0_06GB', '0_003GB', '0_4GB']
Combined genome size (16859816)bp


In [54]:
# Plot the errors:
df_avg = df_summary\
    .groupby("readsGB")[["AE","RAE"]].mean().reset_index()\
    .melt(id_vars="readsGB",value_vars=["AE","RAE"], var_name="Metric")
df_avg.head(2)

,readsGB,Metric,value
0,0.001,AE,0.940687
1,0.002,AE,0.943875


In [115]:
fig = px.line(df_avg, x="readsGB",y="value",line_dash="Metric",
              labels={"value": "Error", "readsGB": "Sample size (Gb)"},
              markers=True,
             title="Average quantification errors<br><sup>Quantification By mapping<sup>")
fig.update_layout(xaxis_range=[0,0.35], yaxis_range=[0,4])

In [72]:
fig = px.line(df_summary.sort_values("readsGB"), x="readsGB",y="RAE", color="Contig",
        labels={"value": "Error", "readsGB": "Sample size (Gb)","Contig":"BGC"},
              markers=True,
             title="Per BGC quantification errors<br><sup>Quantification By mapping<sup>")
fig.update_layout(xaxis_range=[0,0.35], yaxis_range=[0,4])

In [73]:
df_summary.head(1)

,Contig,Breadth,Depth avg,Depth median,readsGB,expected_value,expected_value_int,AE,RAE
0,NC_014328.1.region004,1.0,25.510923,24.0,0.5,29.656314,30.0,5.656314,0.190729


In [104]:
import plotly.graph_objects as go

In [124]:
px.scatter(df_summary, x="readsGB",y="std_res",
            labels={"std_err": "Standardized Residuals", "readsGB": "Sample size (Gb)"},
            title="Residual Plot for each BGC<br><sup>Difference between Median Depth and expected BGC count<sup>"
          )

In [107]:
fig = px.scatter(df_summary, x="readsGB",y="Depth median", 
                 #error_y="err_pos", error_y_minus="err_neg",
                 color="col_standin",
                 labels={"value": "Error", "readsGB": "Sample size (Gb)"})
fig.add_trace(go.Scatter(
    x=df_summary["readsGB"], y=df_summary["expected_value"],
    name = "Expected"
))
#fig.update_layout()